# Paradise Loop - Training file
## AI that generates a 'Paradise Lost'-esque poem

In [1]:
from tensorflow import keras
import numpy as np
import random, sys,os

### Getting the Paradise Lost text data
You can find the file I used at http://www.gutenberg.org/cache/epub/26/pg26.txt <br>
I did a bit of preprocessing, removing the intro text and the licence at the end

In [2]:
path = 'paradise-lost.txt'
text = open(path).read().lower()
print("Corpus length:",len(text))

Corpus length: 453692


### Extracting the training data
The training data is sequences of characters of a given length (in this case 80), and the correct answers are the characters that follow this sequence.

In [3]:
maxlen = 80 # length of sequence (in characters) the model will look at
step = 3 # new sequence every X characters
sentences = []
next_chars = []

for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

# Making a char index
chars = sorted(list(set(text)))
char_index = dict((char, chars.index(char)) for char in chars)

# One-hot-encoding the sequences
x = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)

for i, sentence in enumerate(sentences):
    for j, char in enumerate(sentence):
        x[i,j,char_index[char]] = 1
        y[i,char_index[next_chars[i]]] = 1


### Building the model
In this case, I used two LSTM layers, followed by a Dense output layer.

In [7]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128,input_shape=(maxlen,len(chars)),return_sequences=True,recurrent_dropout=0.3))
model.add(keras.layers.LSTM(64,recurrent_dropout=0.3))
model.add(keras.layers.Dense(len(chars),activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

### Defining a method to sample the next character
The model's output is a probability tensor of size len(chars).<br>
This sampling method gives us more control over which character is picked from this tensor, by using 'temperature'.<br>
A temperature 1 means weighted randomness, so a char with value 0.3 gets picked 30% of the time.
A temperature of 0 means always picking the char with the highest value
This is a spectrum, so try out different values between 0 and 1

In [5]:
def sample(preds,temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    if (temperature == 0):
        # Avoiding a division by 0 error
        return np.argmax(preds)
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds,1)
    return np.argmax(probas)

### Training the model
Here the model is trained<br>
Some output is generated at each epoch, so we can see how to model is doing.<br>
The model can also be saved at every epoch, so that we can go back to better performing models afterwards.

In [9]:
gen_length = 200 # How many chars to generate
save_every = 1 # How often to save the model
temperatures_to_try = [0.2,0.5,1.0] # Which temperatures to showcase during training
                                    # You can use different ones afterwards, this is for monitoring

model_dir = "paradise_loop_models"  # If you change this, make sure to change it in the text generation file too
try:
    os.mkdir(model_dir)
except:
    print("Directory",model_dir,"exists, adding files there")
for epoch in range(1,30):
    print("\n\nEpoch",epoch)
    callbacks = []
    if epoch % save_every == 0:
        callbacks = [keras.callbacks.ModelCheckpoint(filepath=(model_dir+"/epoch_'+str(epoch)+'_2lstms.hdf5"),verbose=0)]
    model.fit(x,y,batch_size=128,epochs=1,callbacks=callbacks)
    
    for temperature in temperatures_to_try:
        print("\nTemperature:",temperature)
        
         # Picking a random starting sequence
        start_i = random.randint(0,len(text)-maxlen-1)
        generated_text = text[start_i:start_i+maxlen]
        print("Generating with starting sequence: \""+generated_text + "\"")
        sys.stdout.write(generated_text)
        for i in range(gen_length):
            # Sampled holds the last MAXLEN characters
            sampled = np.zeros((1,maxlen,len(chars)))
            for j, char in enumerate(generated_text):
                sampled[0,j,char_index[char]] = 1.
            
            preds = model.predict(sampled,verbose=0)[0] # gets the next character (in one-hot-encode form)
            next_index = sample(preds,temperature) # gets index of next character (as int)
            next_char = chars[next_index]
            
            # Updates the 'generated_text' variable
            generated_text = generated_text[1:] + next_char
            
            # Writes the next char
            sys.stdout.write(next_char)



Epoch 1
Epoch 1/1
151204/151204 [==============================] - 231s 2ms/step - loss: 2.1444

Temperature: 0.2
Generating with starting sequence: " nd pile up every stone
of lustre from the brook, in memorie,
or monument to ages "
nd pile up every stone
of lustre from the brook, in memorie,
or monument to ages of the to the sear the sear the sole and the sear the sole the fart the sear the seare the sear the solated the core the sear shall the sear the fart the sore the sole the sate the brie the selm the 
Temperature: 0.5
Generating with starting sequence: " 
glorie and praise, whose wisdom had ordain’d
good out of evil to create, in ste "

glorie and praise, whose wisdom had ordain’d
good out of evil to create, in sterpar stay,
and this with hen sight his light the vall with shall what lid to amper the lowes and deash and which som when subfire the mast discelse the god whom as pire the  the fall with lat resing t
Temperature: 1.0
Generating with starting sequence: " n drie grou

/home/tadeas/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


, or the

Epoch 10
Epoch 1/1
151204/151204 [==============================] - 194s 1ms/step - loss: 1.6330

Temperature: 0.2
Generating with starting sequence: "  art heav’nlie, shee an empty dreame.

say goddess, what ensu’d when raphael,
th "
 art heav’nlie, shee an empty dreame.

say goddess, what ensu’d when raphael,
this seems the seemd the seemd the ground
and the seemd the seemd the seems of heav’n,
and some the seems the seems the seems
the some the seems the sea the seems of heav’n,
and the seems the seemd and 
Temperature: 0.5
Generating with starting sequence: " uldst my firmness therefore doubt
to god or thee, because we have a foe
may temp "
uldst my firmness therefore doubt
to god or thee, because we have a foe
may temper in in heav’n, and the remov’d
all leaded thou persibise, and begin
the second thir spoise of in the god
i many with thir battel first half leose and let the more,
shall and the provids, and sons, t
Temperature: 1.0
Generating with starting sequence: " co


Temperature: 0.2
Generating with starting sequence: "  heard, and hast not fear’d,
but still rejoyc’t, how is it now become
so dreadfu "
 heard, and hast not fear’d,
but still rejoyc’t, how is it now become
so dreadful the seems the self the seems the self the seemd
of the self the self the self and songs of heav’n,
and some the self the self the self the self the self
the self the self the seems the sea the self 
Temperature: 0.5
Generating with starting sequence: "  shrub,
and bush with frizl’d hair implicit: last
rose as in dance the stately t "
 shrub,
and bush with frizl’d hair implicit: last
rose as in dance the stately thou the bliss of heav’n;
the with else the power and then the swarmd
the self and in the power of heav’n, though the shalt with grow
of heav’n new world remove works thee seemd
cronged in the earth th
Temperature: 1.0
Generating with starting sequence: " uce new worlds; whereof so rife
there went a fame in heav’n that he ere long
int "
uce new worlds; whereof so

equal in days and not the sease the seat
the sent and starr the seat the seat
the send and sun the stor’d the sease,
and so stand the seat the seat, and the seat
the sent the seat the seat the seat the sent
to the son t
Temperature: 0.5
Generating with starting sequence: " delay to heare thee tell
his generation, and the rising birth
of nature from the "
delay to heare thee tell
his generation, and the rising birth
of nature from the spirits thir chances of the earth
she spake, and the son, but the seat
of shall the stright and the beast your seat of the wing
with his gast of the confernal seems,
whose not starr of his in the day
Temperature: 1.0
Generating with starting sequence: " rs,
vvith other echo farr i taught your shades
to answer, and resound farr other "
rs,
vvith other echo farr i taught your shades
to answer, and resound farr others, also tright
harpuc, and spak’t; and the his peruent.

on thou the many abichs who man
obpras’d
fell mains uncoke as free.  thou to pay,
not wa